### Keeping your API keys secret

Often, to access data from APIs, you must use an API "key". This key identifies your account to the API you're calling. 

You **never** want to commit your API keys for any service to `git`, especially if your repo is public. Anyone reading your code could take that API key and make API requests on your behalf. If your key is tied to a service like Amazon Web Services, where you can create resources (*that cost money*) using the API, making your key public can be a costly mistake.

Since you want to commit your _code_ to `git`, this means you need a way to separate your code from your API key.

Typically, you'll want to keep your API key in some file on your computer (for some services, it may make sense to encrypt your API key, which is beyond the scope of this document). Below, we'll run through an example of how to pull an API key from a file into a Python program, as a string that you can use in your program.

Before you run the code below, 

* Create a file called `apikey`, with a single line that contains the API key you want to store
* Create a `.gitignore` file, add a single line with the word "apikey" and save the file. `.gitignore` is a special file that `git` uses to ignore any files listed. You won't be asked to add / commit changes to these files, so it's an easy way to make sure you don't accidentally commit these files to `git`. See [this reference](https://git-scm.com/docs/gitignore) for more information.

In [ ]:
# See above - we've created an API key in a file called 'apikey', 
# in the same directory as this notebook
filename = 'apikey'

In [ ]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [ ]:
api_key = get_file_contents(filename)
print("Our API key is: %s" % (api_key))

### Storing more than one piece of data in a config file

Often, you're not just storing one API key or password in a config file. You might have authorization information for multiple sites (OpenWeatherMap API, Twitter, etc.), and you don't want to create different files for each.

You could keep the authorization data for each API on their own lines, like so:

```
123
abcdef
ghijkl
...
```

But there's no easy way to identify what keys correspond to what services. It's just a bunch of meaningless lines in a file, so it's impossible to read.

There are a few ways to solve this problem, but [YAML](https://en.wikipedia.org/wiki/YAML) provides a structured data format that's easy for humans and machines to read. YAML is often used to structure config files like this.

Let's take a look at our example YAML file, `auth.yaml`:

```
twitter:
    consumer_key: your_consumer_key_goes_here
    consumer_secret: your_consumer_secret_goes_here
    access_token: your_access_token_goes_here
    access_token_secret: your_access_token_secret_goes_here
```

First, you'll want to fill in your credentials in place of the example values.

Next, let's use the `yaml` package to read the contents of this file into our Python script. Make sure to `source activate` your virtual environment before you run the `pip` command below:

In [1]:
!pip install pyyaml

Note that the package name we use to install this module is `pyyaml`, but we `import yaml` below.

Here's how we'll use the `yaml` module to read in data from our config file:

In [10]:
import yaml
TWITTER_CONFIG_FILE = 'auth.yaml'

with open(TWITTER_CONFIG_FILE, 'r') as config_file:
    config = yaml.load(config_file)
    
print(type(config))

<class 'dict'>


We see that our `config` variable - the contents of the YAML file - is a dictionary. Let's see what's inside

In [13]:
import json

print(json.dumps(config, indent=4, sort_keys=True))

{
    "twitter": {
        "access_token": "your_access_token_goes_here",
        "access_token_secret": "your_access_token_secret_goes_here",
        "consumer_key": "your_consumer_key_goes_here",
        "consumer_secret": "your_consumer_secret_goes_here"
    }
}


This dictionary looks a lot like the structure of the YAML file itself!

```
twitter:
    consumer_key: your_consumer_key_goes_here
    consumer_secret: your_consumer_secret_goes_here
    access_token: your_access_token_goes_here
    access_token_secret: your_access_token_secret_goes_here
```

You should read more about YAML on Google to understand this structure, and more.

Since this is a dictionary, we can extract our consumer key directly:

In [15]:
consumer_key = config['twitter']['consumer_key']
print(consumer_key)

your_consumer_key_goes_here


**Important Note**: I've only checked the `auth.yaml` file into `git` to show you what a YAML file looks like. As we did above, **remember to add your config file to your `.gitignore` file so that you do not accidentally commit it to `git`**.